In [86]:
with open('16.txt', 'r') as f:
    data = f.read()

In [79]:
data = """class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12"""

In [27]:
def parse(data):
    fields, mine, nearby = data.strip().split('\n\n')
    ranges = {}
    for field in fields.split('\n'):
        i = field.index(':')
        name = field[:i]
        rs = [parts.split('-') for parts in field[i+2:].split(' or ')]
        ranges[name] = [int(i) for r in rs for i in r]
    mine = [int(i) for i in mine.split('\n')[1].split(',')]
    
    nearby = nearby.split('\n')[1:]
    nearby = [[int(i) for i in n.split(',')] for n in nearby]
    
    return ranges, mine, nearby
    

In [28]:
parse(data)

({'class': [1, 3, 5, 7], 'row': [6, 11, 33, 44], 'seat': [13, 40, 45, 50]},
 [7, 1, 14],
 [[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]])

In [29]:
def find_invalid(ranges, mine, nearby):
    error_rate = 0
    for ticket in nearby:
        for entry in ticket:
            for from1, to1, from2, to2 in ranges.values():
                if (from1 <= entry <= to1) or (from2 <= entry <= to2):
                    break
            else:
                error_rate += entry
    return error_rate
    
#     def _valid(vals, x):
#         from1,to1,from2,to2 = vals
#         return (from1 <= x <= to1) or (from2 <= x <= to2)

In [32]:
find_invalid(*parse(data))

23122

In [34]:
1 < np.array([1,2,3]) < 3

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [87]:
def find_permutation(ranges, mine, nearby):
    # Filter invalid
    invalid = set()
    for i, ticket in enumerate(nearby):
        for entry in ticket:
            for from1, to1, from2, to2 in ranges.values():
                if (from1 <= entry <= to1) or (from2 <= entry <= to2):
                    break
            else:
                invalid.add(i)
    nearby = [ticket for i, ticket in enumerate(nearby) if i not in invalid]
    nearby = np.array(nearby)
    N = len(ranges)
    M = len(mine)
    field_matches_column = np.zeros((N,M),dtype=bool)
    fields = [field for field in ranges]
    
    for i, field in enumerate(fields):
        for j in range(M):
            from1, to1, from2, to2 = ranges[field]
            valid = np.all(np.logical_or(
                np.logical_and(from1 <= nearby[:,j], nearby[:,j] <= to1),
                np.logical_and(from2 <= nearby[:,j], nearby[:,j] <= to2),
            ))
            field_matches_column[i,j] = valid
    
    mapping = {}
    for _ in range(N):
        possibilities = field_matches_column.sum(axis=1)
        i = list(possibilities).index(1)
        j = list(field_matches_column[i,:]).index(True)
        field_matches_column[:,j] = False
        mapping[fields[i]] = j
    print(mapping)
    return np.prod([mine[mapping[f]] for f in fields if f.startswith('departure')])
#     print(np.argwhere(possibilites==1)[0][0])
           

In [111]:
def find_permutation(ranges, mine, nearby):
    # Filter invalid
    invalid = set()
    for i, ticket in enumerate(nearby):
        for entry in ticket:
            for from1, to1, from2, to2 in ranges.values():
                if (from1 <= entry <= to1) or (from2 <= entry <= to2):
                    break
            else:
                invalid.add(i)
    nearby = [ticket for i, ticket in enumerate(nearby) if i not in invalid]
    nearby = np.array(nearby)
    N = len(ranges)
    M = len(mine)
    field_matches_column = np.zeros((N,M),dtype=bool)
    fields = [field for field in ranges]
    
    for i, field in enumerate(fields):
        for j in range(M):
            from1, to1, from2, to2 = ranges[field]
            valid = np.all(np.logical_or(
                np.logical_and(from1 <= nearby[:,j], nearby[:,j] <= to1),
                np.logical_and(from2 <= nearby[:,j], nearby[:,j] <= to2),
            ))
            field_matches_column[i,j] = valid
    
    mapping = {}
    for _ in range(N):
        possibilities = field_matches_column.sum(axis=1)
        i = list(possibilities).index(1)
        j = list(field_matches_column[i,:]).index(True)
        field_matches_column[:,j] = False
        mapping[fields[i]] = j
    print(mapping)
    return np.prod([mine[mapping[f]] for f in fields if f.startswith('departure')])
#     print(np.argwhere(possibilites==1)[0][0])
           

In [112]:
def find_permutation(ranges, mine, nearby):
    # Filter invalid
    invalid = set()
    for i, ticket in enumerate(nearby):
        for entry in ticket:
            for from1, to1, from2, to2 in ranges.values():
                if (from1 <= entry <= to1) or (from2 <= entry <= to2):
                    break
            else:
                invalid.add(i)
    nearby = [ticket for i, ticket in enumerate(nearby) if i not in invalid]
    nearby = np.array(nearby)
    N = len(ranges)
    M = len(mine)
    field_matches_column = np.zeros((N,M),dtype=bool)
    fields = [field for field in ranges]
    
    for i, field in enumerate(fields):
        for j in range(M):
            from1, to1, from2, to2 = ranges[field]
            valid = np.all(np.logical_or(
                np.logical_and(from1 <= nearby[:,j], nearby[:,j] <= to1),
                np.logical_and(from2 <= nearby[:,j], nearby[:,j] <= to2),
            ))
            field_matches_column[i,j] = valid
    
    mapping = {}
#     plt.imshow(field_matches_column.astype(int));plt.show()
    field_order = np.argsort(field_matches_column.sum(axis=1))
    field_matches_column = field_matches_column[field_order,:]
#     plt.imshow(field_matches_column.astype(int));plt.show()
    column_order = np.argsort(field_matches_column.sum(axis=0))
    field_matches_column = field_matches_column[:,column_order]
#     plt.imshow(field_matches_column.astype(int));plt.show()
    print(field_order)
    print(column_order)
    mapping = {fields[i]:j for i,j in zip(field_order, reversed(column_order))}
    print(mapping)
    return np.prod([mine[mapping[f]] for f in fields if f.startswith('departure')])
#     print(np.argwhere(possibilites==1)[0][0])
           

In [113]:
find_permutation(*parse(data))

[15 19 13 16 18  3  2  1  5  4  0  7 12  8  6 17 10 11 14  9]
[13  8 18  5  0 15 10 16  3  1 14  6 19  2  4 12  7 17  9 11]
{'seat': 11, 'zone': 9, 'route': 17, 'train': 7, 'wagon': 12, 'departure track': 4, 'departure platform': 2, 'departure station': 19, 'departure time': 6, 'departure date': 14, 'departure location': 1, 'arrival station': 3, 'price': 16, 'arrival platform': 10, 'arrival location': 15, 'type': 0, 'class': 5, 'duration': 18, 'row': 8, 'arrival track': 13}


362974212989

In [64]:
sorted([11, 8, 7, 6, 10, 9, 15, 12, 14, 20, 17, 18, 13, 3, 19, 1, 4, 16, 5, 2])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [65]:
[11, 8, 7, 6, 10, 9, 15, 12, 14, 20, 17, 18, 13, 3, 19, 1, 4, 16, 5, 2][1]

8